In [ ]:
import pandas as pd
import requests

import json
import time
import random

In [2]:
# default page 
rf = requests.get('https://open.umn.edu/opentextbooks/textbooks.json?page=1')

In [54]:
rf.json()['links']

{'self': 'https://open.umn.edu/opentextbooks/textbooks.json?page=1',
 'total_pages': 165,
 'total_count': 1642,
 'next': 'https://open.umn.edu/opentextbooks/textbooks.json?page=2'}

In [74]:
df = pd.json_normalize(rf.json()['data'], max_level=0)
print('Info: ', df.info())
df.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      10 non-null     int64 
 1   title                   10 non-null     object
 2   edition_statement       2 non-null      object
 3   volume                  1 non-null      object
 4   copyright_year          10 non-null     int64 
 5   ISBN10                  0 non-null      object
 6   ISBN13                  6 non-null      object
 7   license                 10 non-null     object
 8   language                10 non-null     object
 9   description             10 non-null     object
 10  contributors            10 non-null     object
 11  subjects                10 non-null     object
 12  publishers              10 non-null     object
 13  formats                 10 non-null     object
 14  rating                  0 non-null      object
 15  textbook_

,id,title,edition_statement,volume,copyright_year,ISBN10,ISBN13,license,language,description,contributors,subjects,publishers,formats,rating,textbook_reviews_count,reviews,url,updated_at
0,1709,Applications of Educational Technology,None,None,2018,None,None,Attribution-NonCommercial,eng,Welcome to the Applications of Educational Tec...,"[{'id': 7214, 'contribution': 'Author', 'prima...","[{'id': 5, 'name': 'Education', 'parent_subjec...","[{'id': 1683, 'name': 'Oklahoma State Universi...","[{'id': 4357, 'format': 'Online', 'url': 'http...",None,0,[],https://open.umn.edu/opentextbooks/textbooks/a...,2024-08-02T04:30:37.000Z


In [124]:
dfc = df.copy()
for pn in range(2, rf.json()['links']['total_pages']+1):
    r = requests.get(f'https://open.umn.edu/opentextbooks/textbooks.json?page={pn}', timeout=4)
    dfc = pd.concat([dfc, pd.json_normalize(r.json()['data'], max_level=0)], axis=0, ignore_index=True)
    time.sleep(random.randint(1, 3))

In [130]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1524 entries, 0 to 1523
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1524 non-null   int64  
 1   title                   1524 non-null   object 
 2   edition_statement       210 non-null    object 
 3   volume                  49 non-null     object 
 4   copyright_year          1492 non-null   float64
 5   ISBN10                  19 non-null     object 
 6   ISBN13                  633 non-null    object 
 7   license                 1524 non-null   object 
 8   language                1524 non-null   object 
 9   description             1524 non-null   object 
 10  contributors            1524 non-null   object 
 11  subjects                1524 non-null   object 
 12  publishers              1524 non-null   object 
 13  formats                 1524 non-null   object 
 14  rating                  1049 non-null   

In [146]:
# Some duplicated books
(dfc['id'].value_counts() > 1).any()

True

In [148]:
dfc.drop_duplicates('id', inplace=True)

In [149]:
(dfc
 .set_index('id')
 .to_json('Data\OTL_textbooks_v0.json', orient='index')
)